### 7.9 用函数替代只有单个方法地类

我们有一个只定义了一个方法地类（除了__init__()方法外）。但是为了简化代码，我们更希望能用一个简单地函数替代。

通常情况下，只有单个方法地类可以通过闭包（closure）将其转换成函数。

In [2]:
from urllib.request import urlopen

class UrlTemplate:
    def __init__(self, template):
        self.template = template
    def open(self, **kwargs):
        return urlopen(self.template.format_map(kwargs))
    
# # Example use. Download stock data from yahoo
# yahoo = UrlTemplate('http://finace.yahoo.com/d/quotes.csv?s={names}&f={fields}')
# for line in yahoo.open(names='IBM,AAPL,FB', fields='sllclv'):
#    print(line.decode('utf-8'))

这个类可以用一个简单地函数来取代：

In [ ]:
def  urltemplate(template):
    def opener(**kwargs):
        return urlopen(template.format_map(kwargs))
    return opener

# # Example use
# yahoo = urltemplate('http://finace.yahoo.com/d/quotes.csv?s={names}&f={fields}')
# for line in yahoo(names='IBM,AAPL,FB', fields='sllclv'):
#     print(line.decode('utf-8'))

### 7.10 在回调函数中携带额外地状态

In [19]:
def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)
    
    # Invoke the callback with the result
    return callback(result)
    
def print_result(result):
    print('Got:', result)
    
def add(x, y):
    return x + y

apply_async(add, (2, 3), callback=print_result)
apply_async(add, ("hello ", "world"), callback=print_result)

Got: 5
Got: hello world


我们注意到函数print_result()仅接受一个单独的参数，也就是result。  
有时我们希望回调函数也可以同其他变量或者部分环境就行交互。  
一种在回调函数中携带额外信息的方法是使用绑定方法（bound-method)而不是普通的函数。

下面这个类保存了一个内部的序列号码，每当接收到一个结果时就递增这个号码。

In [5]:
class ResultHandler:
    def __init__(self):
        self.sequence = 0
    def handler(self, result):
        self.sequence += 1
        print('[{}] Got: {}'.format(self.sequence, result))
        
r = ResultHandler()
apply_async(add, (2, 3), callback=r.handler)
apply_async(add, ("hello ", "world"), callback=r.handler)

[1] Got: 5
[2] Got: hello world


作为类的替代方案，也可以使用闭包来捕获状态。

In [20]:
def make_handler():
    sequence = 0
    def handler(result):
        nonlocal sequence
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))
    return handler

handler = make_handler()
apply_async(add, (2, 3), callback=handler)
apply_async(add, ("hello ", "world"), callback=handler)

[1] Got: 5
[2] Got: hello world


也可以用协程（coroutine）来完成这个任务。

In [28]:
def make_handler():
    sequence = 0
    while True:
        result = yield
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))

handler = make_handler()
next(handler) # Advance to the yield
apply_async(add, (2, 3), callback=handler.send)
apply_async(add, ("hello ", "world"), callback=handler.send)

[1] Got: 5
[2] Got: hello world


也可以通过额外的参数在回调函数中携带状态，然后用partial()来处理参数个数问题。

In [29]:
class SequenceNo:
    def __init__(self):
        self.sequence = 0
        
def handler(result, seq):
    seq.sequence += 1
    print('[{}] Got: {}'.format(seq.sequence, result))
    
seq = SequenceNo()
from functools import partial

apply_async(add, (2, 3), callback=partial(handler, seq=seq))
apply_async(add, ("hello ", "world"), callback=partial(handler, seq=seq))

[1] Got: 5
[2] Got: hello world


### 7.11 内联回调函数

假设有一个函数会按如下方式调用回调函数

In [ ]:
def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)
    
    # Invoke the callback with the result
    callback(result)

内联的版本如下：

In [30]:
from queue import Queue
from functools import wraps

class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args


def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func, a.args, callback=result_queue.put)
            except StopIteration:
                break
    return wrapper

# 我们通过yield语句将回调函数变为内联的
def add(x, y):
    return x + y

@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)
    r = yield Async(add, ('hello ', 'world'))
    print(r)
    for n in range(10):
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')
    
test()

5
hello world
0
2
4
6
8
10
12
14
16
18
Goodbye


### 7.12 访问定义在闭包内的变量

一般来说，在闭包内层定义的变量对于外界来说是完全隔离的。但是，可以通过编写存取函数（accessor function， 即 getter/setter方法）并将他们作为函数属性附加到闭包上来提供对内层变量的访问支持。

In [31]:
def sample():
    n = 0
    # Closure function
    def func():
        print('n= ', n)
        
    # Accessor methods for n
    def get_n():
        return n
    
    def set_n(value):
        nonlocal n
        n = value
        
    # Attach as function attriutes
    func.get_n = get_n
    func.set_n = set_n
    
    return func

In [32]:
f = sample()
f()

n=  0


In [33]:
f.set_n(10)

In [34]:
f.get_n()

10

我们对本节的技术稍做扩展就可以让闭包模拟成类实例。我们要做的就是将内层函数拷贝到一个实例的字典中然后将它返回。

In [46]:
import sys
class ClosureInstance:
    def __init__(self, local=None):
        if local is None:
            local = sys._getframe(1).f_locals
            
        # Update instance dictionary with callables
        self.__dict__.update((key, value) for key, value in local.items() 
                            if callable(value))
        
    # Redirect special methods
    def __len__(self):
        return self.__dict__['__len__']()
    

# Example use
def Stack():
    items = []
    
    def push(item):
        items.append(item)
        
    def pop():
        return items.pop()
    
    def __len__():
        return len(items)
    return ClosureInstance()

In [47]:
s = Stack()
print(s)

In [48]:
s.push(10)
s.push(20)
s.push('Hello')
len(s)

3